# Datasets can be downloaded from UNECE website: http://www.unece.org/cefact/codesfortrade/codes_index.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
locode_part1 = pd.read_csv("datasets/2019-2 UNLOCODE CodeListPart1.csv", encoding = 'unicode_escape', header=None)
locode_part2 = pd.read_csv("datasets/2019-2 UNLOCODE CodeListPart2.csv", encoding = 'unicode_escape', header=None)
locode_part3 = pd.read_csv("datasets/2019-2 UNLOCODE CodeListPart3.csv", encoding = 'unicode_escape', header=None)

locode_df = pd.concat([locode_part1, locode_part2, locode_part3])

locode_df.columns = ['change', 'country', 'location', 'name', 'name_wo_diacritics', 'subdivision', 'status', 'function', 'date', 'iata', 'coordinates', 'remarks' ]

In [3]:
locode_df.head(10)

,change,country,location,name,name_wo_diacritics,subdivision,status,function,date,iata,coordinates,remarks
0,NaN,AD,NaN,.ANDORRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AD,ALV,Andorra la Vella,Andorra la Vella,NaN,--34-6--,AI,601.0,NaN,4230N 00131E,NaN
2,NaN,AD,CAN,Canillo,Canillo,NaN,--3-----,RL,307.0,NaN,4234N 00135E,NaN
3,NaN,AD,ENC,Encamp,Encamp,NaN,--3-----,RL,307.0,NaN,4232N 00134E,NaN
4,NaN,AD,ESC,Escaldes-Engordany,Escaldes-Engordany,NaN,--3-----,RL,307.0,NaN,4231N 00133E,NaN
5,NaN,AD,EAC,Escàs,Escas,04,--3-----,RL,1407.0,NaN,4233N 00131E,NaN
6,NaN,AD,FMO,La Farga de Moles,La Farga de Moles,NaN,--3----B,RQ,307.0,NaN,NaN,NaN
7,NaN,AD,LMA,La Massana,La Massana,NaN,--3-----,RL,307.0,NaN,4234N 00129E,NaN
8,NaN,AD,ORD,Ordino,Ordino,NaN,--3-----,RL,307.0,NaN,4233N 00131E,NaN
9,NaN,AD,PAS,Pas de la Casa,Pas de la Casa,NaN,--3----B,RL,307.0,NaN,4233N 00144E,NaN


# Create DataFrame with countries only




In [4]:
country_condition = locode_df['location'].isnull() & locode_df['change'].isnull()

countries_df = locode_df[country_condition]
countries_df = countries_df[["country", "name"]]
countries_df = countries_df.set_index('country')
countries_df['name'] = countries_df['name'].str.slice(start=1)
countries_df = countries_df.rename(columns={"name": "country_name"})

In [6]:
countries_df.head(10)

,country_name
country,
AD,ANDORRA
AE,UNITED ARAB EMIRATES
AF,AFGHANISTAN
AG,ANTIGUA AND BARBUDA
AI,ANGUILLA
AL,ALBANIA
AM,ARMENIA
AO,ANGOLA
AQ,ANTARCTICA


# Create DataFrame with ports only

In [7]:
ports_df = locode_df[locode_df["status"].str.slice(start=0, stop=1) == '1'].drop(columns=['change', 'status', 'date', 'remarks', 'function', 'iata', 'subdivision'])

ports_df = ports_df.dropna(subset=['coordinates']) 

ports_df.loc[:, "latitude"] = ports_df.coordinates.str.split(expand=True).loc[:, 0]
ports_df.loc[:, "longitude"] = ports_df.coordinates.str.split(expand=True).loc[:, 1]

ports_df["latitude"] = (ports_df["latitude"].str.slice(0, 2).astype("double") + ports_df["latitude"].str.slice(2, 4).astype("double") / 60) * ports_df["latitude"].str.slice(4, 5).map(lambda x: 1 if x == "N" else -1)
ports_df["longitude"] = (ports_df["longitude"].str.slice(0, 3).astype("double") + ports_df["longitude"].str.slice(3, 5).astype("double") / 60) * ports_df["longitude"].str.slice(5, 6).map(lambda x: 1 if x == "E" else -1)

ports_df = ports_df.drop(columns=["coordinates"])

In [8]:
ports_df.head(10)

,country,location,name,name_wo_diacritics,latitude,longitude
13,AE,ABU,Abu al Bukhoosh,Abu al Bukhoosh,25.483333,53.133333
14,AE,AUH,Abu Dhabi,Abu Dhabi,24.466667,54.366667
15,AE,AMU,Abu Musa,Abu Musa,25.866667,55.016667
16,AE,ARP,Ahmed Bin Rashid Port,Ahmed Bin Rashid Port,25.533333,55.550000
21,AE,FJR,Al Fujayrah,Al Fujayrah,25.116667,56.333333
23,AE,JAZ,Al Jazeera Port,Al Jazeera Port,25.716667,55.800000
24,AE,AJP,Al Jeer Port,Al Jeer Port,26.033333,56.050000
28,AE,RUW,Ar Ruways,Ar Ruways,24.116667,52.733333
32,AE,DBP,Dibba,Dibba,25.616667,56.266667
33,AE,DXB,Dubai,Dubai,25.250000,55.266667


In [9]:
ports_with_country_names_df = ports_df.join(countries_df, how='left', on="country")
ports_with_country_names_df = ports_with_country_names_df.reset_index()
ports_with_country_names_df = ports_with_country_names_df.drop(columns=["index"])

ports_with_country_names_df

,country,location,name,name_wo_diacritics,latitude,longitude,country_name
0,AE,ABU,Abu al Bukhoosh,Abu al Bukhoosh,25.483333,53.133333,UNITED ARAB EMIRATES
1,AE,AUH,Abu Dhabi,Abu Dhabi,24.466667,54.366667,UNITED ARAB EMIRATES
2,AE,AMU,Abu Musa,Abu Musa,25.866667,55.016667,UNITED ARAB EMIRATES
3,AE,ARP,Ahmed Bin Rashid Port,Ahmed Bin Rashid Port,25.533333,55.550000,UNITED ARAB EMIRATES
4,AE,FJR,Al Fujayrah,Al Fujayrah,25.116667,56.333333,UNITED ARAB EMIRATES
...,...,...,...,...,...,...,...
18022,ZW,CHE,Chegutu,Chegutu,-18.133333,30.150000,ZIMBABWE
18023,ZW,FLU,Filabusi,Filabusi,-20.533333,29.283333,ZIMBABWE
18024,ZW,GWA,Gwanda,Gwanda,-20.933333,29.000000,ZIMBABWE
18025,ZW,RSP,Rusape,Rusape,-18.533333,32.116667,ZIMBABWE


In [10]:
ports_with_country_names_df.to_csv("unlocode_ports.csv", index=False)